In [1]:
## Importing Packages

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
import pickle

[nltk_data] Downloading package punkt to /Users/ayaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ayaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
## Reading the data

data = pd.read_csv("training.csv", encoding="ISO-8859-1")

In [6]:
data.head()

,text,category
0,I have been waiting over a week. Is the card s...,Negative Customer Experience
1,"How do I know if I will get my card, or if it ...",Negative Customer Experience
2,My new card has not been delivered to my home ...,Negative Customer Experience
3,"Last Saturday, I was charged extra for the exc...",Negative Customer Experience
4,"Hi, I am disappointed to see such a bad exchan...",Negative Customer Experience


## Data Cleaning 

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4712 entries, 0 to 4711
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      4712 non-null   object
 1   category  4712 non-null   object
dtypes: object(2)
memory usage: 73.8+ KB


In [8]:
### Encoding the target values - ham to 0 and Spam to 1

encoder = LabelEncoder()
data['category'] = encoder.fit_transform(data['category'])

In [9]:
data['category'].unique()

array([2, 5, 0, 1, 3, 4])

In [11]:
# Get the mapping of labels to encoded values
label_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

# Print the label mapping
print(label_mapping)

{'Account Enquiry': 0, 'Account Update': 1, 'Negative Customer Experience': 2, 'Positive Customer Review': 3, 'Spam': 4, 'Usage Issue': 5}


In [12]:
data

,text,category
0,I have been waiting over a week. Is the card s...,2
1,"How do I know if I will get my card, or if it ...",2
2,My new card has not been delivered to my home ...,2
3,"Last Saturday, I was charged extra for the exc...",2
4,"Hi, I am disappointed to see such a bad exchan...",2
...,...,...
4707,Want explicit SEX in 30 secs? Ring 02073162414...,4
4708,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,4
4709,Had your contract mobile 11 Mnths? Latest Moto...,4
4710,REMINDER FROM O2: To get 2.50 pounds free call...,4


In [13]:
## Preprocessing 
# - Checking for missing values
# - Checking for duplicates and keeping only one

def preprocess(data):
    print("Null Values: ", data.isnull().sum())
    data = data.drop_duplicates(keep='first')
    print("Duplicate data: ", data.duplicated().sum())
    print("Shape : ", data.shape)
    print("Target Variable Value Count:", data['category'].value_counts())
    
preprocess(data)

Null Values:  text        0
category    0
dtype: int64
Duplicate data:  0
Shape :  (4614, 2)
Target Variable Value Count: 5    2949
0     599
2     464
3     318
4     163
1     121
Name: category, dtype: int64


In [14]:
data['num_characters'] = data['text'].apply(len)

In [15]:
data['num_of_words'] = data['text'].apply(lambda x: len(nltk.word_tokenize(x)))

In [16]:
data['num_of_sentences'] = data['text'].apply(lambda x: len(nltk.sent_tokenize(x)))

In [17]:
data.head()

,text,category,num_characters,num_of_words,num_of_sentences
0,I have been waiting over a week. Is the card s...,2,58,14,2
1,"How do I know if I will get my card, or if it ...",2,54,17,1
2,My new card has not been delivered to my home ...,2,68,17,2
3,"Last Saturday, I was charged extra for the exc...",2,91,19,2
4,"Hi, I am disappointed to see such a bad exchan...",2,155,33,2


In [18]:
data

,text,category,num_characters,num_of_words,num_of_sentences
0,I have been waiting over a week. Is the card s...,2,58,14,2
1,"How do I know if I will get my card, or if it ...",2,54,17,1
2,My new card has not been delivered to my home ...,2,68,17,2
3,"Last Saturday, I was charged extra for the exc...",2,91,19,2
4,"Hi, I am disappointed to see such a bad exchan...",2,155,33,2
...,...,...,...,...,...
4707,Want explicit SEX in 30 secs? Ring 02073162414...,4,90,18,3
4708,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,4,160,38,6
4709,Had your contract mobile 11 Mnths? Latest Moto...,4,160,35,5
4710,REMINDER FROM O2: To get 2.50 pounds free call...,4,147,30,1


## Text Preprocessing

In [19]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)
            
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))
    
            
    return " ".join(y)

In [20]:
transform_text("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.")

'gon na home soon want talk stuff anymor tonight k cri enough today'

In [21]:
data['transformed_text'] = data['text'].apply(transform_text)

In [22]:
data.head()

,text,category,num_characters,num_of_words,num_of_sentences,transformed_text
0,I have been waiting over a week. Is the card s...,2,58,14,2,wait week card still come
1,"How do I know if I will get my card, or if it ...",2,54,17,1,know get card lost
2,My new card has not been delivered to my home ...,2,68,17,2,new card deliv home yet go
3,"Last Saturday, I was charged extra for the exc...",2,91,19,2,last saturday charg extra exchang rate payment...
4,"Hi, I am disappointed to see such a bad exchan...",2,155,33,2,hi disappoint see bad exchang rate hope assist...


In [23]:
#from wordcloud import WordCloud 
#wc = WordCloud(width=50, height=50, min_font_size=10, background_color='white')

In [24]:
#wc.generate(data[data['target'] == 1]['transformed_text'].str.cat(sep=""))

## Model Building

In [25]:
## Converting text to numbers

cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)

In [26]:
X = tfidf.fit_transform(data['transformed_text']).toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
X.shape

(4712, 2602)

In [28]:
y = data['category'].values

In [29]:
y

array([2, 2, 2, ..., 4, 4, 4])

In [30]:
y.shape

(4712,)

In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [32]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [33]:
mnb.fit(X_train,y_train)
y_pred2 = mnb.predict(X_test)
#print(accuracy_score(y_test,y_pred2,average='micro'))
#print(confusion_matrix(y_test,y_pred2,average='micro'))
print(precision_score(y_test,y_pred2,average='micro'))

0.8176033934252386


In [35]:
pickle.dump(tfidf,open('vectorizer.pkl','wb'))


pickle.dump(mnb,open('modelFinal.pkl','wb'))

In [36]:
data

,text,category,num_characters,num_of_words,num_of_sentences,transformed_text
0,I have been waiting over a week. Is the card s...,2,58,14,2,wait week card still come
1,"How do I know if I will get my card, or if it ...",2,54,17,1,know get card lost
2,My new card has not been delivered to my home ...,2,68,17,2,new card deliv home yet go
3,"Last Saturday, I was charged extra for the exc...",2,91,19,2,last saturday charg extra exchang rate payment...
4,"Hi, I am disappointed to see such a bad exchan...",2,155,33,2,hi disappoint see bad exchang rate hope assist...
...,...,...,...,...,...,...
4707,Want explicit SEX in 30 secs? Ring 02073162414...,4,90,18,3,want explicit sex 30 sec ring 02073162414 cost...
4708,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,4,160,38,6,ask 3mobil 0870 chatlin inclu free min india c...
4709,Had your contract mobile 11 Mnths? Latest Moto...,4,160,35,5,contract mobil 11 mnth latest motorola nokia e...
4710,REMINDER FROM O2: To get 2.50 pounds free call...,4,147,30,1,remind o2 get pound free call credit detail gr...


In [9]:
import numpy as np
import pandas as pd
from datetime import datetime
dates = []
def generate_random_dates(start_date, end_date, k):
   date_range = (end_date - start_date).days + 1
   random_days = np.random.randint(date_range, size=k)
   random_dates = pd.to_datetime(start_date) + pd.to_timedelta(random_days, unit='d')
   return random_dates

start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 10, 1)
print("The random dates generated are:")
random_dates = generate_random_dates(start_date, end_date, 5572)
for index,date in enumerate(random_dates):
    print(f"{index+1}. {date.strftime('%Y-%m-%d')}")
    dates.append(date.strftime('%Y-%m-%d'))
    

The random dates generated are:
1. 2023-05-04
2. 2023-03-12
3. 2023-08-29
4. 2023-03-09
5. 2023-02-10
6. 2023-02-16
7. 2023-09-08
8. 2023-05-30
9. 2023-07-28
10. 2023-02-09
11. 2023-05-23
12. 2023-04-24
13. 2023-02-23
14. 2023-02-16
15. 2023-06-09
16. 2023-05-17
17. 2023-03-01
18. 2023-09-04
19. 2023-01-03
20. 2023-03-19
21. 2023-08-03
22. 2023-02-17
23. 2023-05-11
24. 2023-03-15
25. 2023-05-29
26. 2023-02-01
27. 2023-08-19
28. 2023-01-30
29. 2023-08-02
30. 2023-06-04
31. 2023-02-16
32. 2023-05-23
33. 2023-05-28
34. 2023-04-20
35. 2023-09-05
36. 2023-03-29
37. 2023-08-15
38. 2023-01-24
39. 2023-05-22
40. 2023-07-09
41. 2023-05-31
42. 2023-09-16
43. 2023-09-09
44. 2023-04-28
45. 2023-02-08
46. 2023-04-23
47. 2023-03-15
48. 2023-07-16
49. 2023-03-25
50. 2023-08-12
51. 2023-09-14
52. 2023-03-14
53. 2023-06-28
54. 2023-06-17
55. 2023-06-14
56. 2023-07-20
57. 2023-07-28
58. 2023-09-14
59. 2023-03-15
60. 2023-05-20
61. 2023-08-23
62. 2023-03-28
63. 2023-05-07
64. 2023-04-24
65. 2023-03-16
66

1844. 2023-06-07
1845. 2023-04-25
1846. 2023-03-27
1847. 2023-09-29
1848. 2023-03-29
1849. 2023-03-25
1850. 2023-05-24
1851. 2023-06-19
1852. 2023-05-31
1853. 2023-09-13
1854. 2023-07-16
1855. 2023-06-03
1856. 2023-08-05
1857. 2023-03-23
1858. 2023-06-09
1859. 2023-08-19
1860. 2023-03-06
1861. 2023-08-25
1862. 2023-09-11
1863. 2023-08-24
1864. 2023-05-15
1865. 2023-03-23
1866. 2023-04-25
1867. 2023-08-14
1868. 2023-04-16
1869. 2023-06-24
1870. 2023-07-07
1871. 2023-02-06
1872. 2023-09-25
1873. 2023-09-06
1874. 2023-04-09
1875. 2023-04-26
1876. 2023-05-27
1877. 2023-03-22
1878. 2023-04-15
1879. 2023-08-10
1880. 2023-01-31
1881. 2023-08-10
1882. 2023-01-26
1883. 2023-04-09
1884. 2023-08-07
1885. 2023-08-22
1886. 2023-09-03
1887. 2023-07-08
1888. 2023-09-03
1889. 2023-08-29
1890. 2023-03-01
1891. 2023-02-10
1892. 2023-01-06
1893. 2023-03-11
1894. 2023-05-28
1895. 2023-04-07
1896. 2023-03-04
1897. 2023-03-04
1898. 2023-06-23
1899. 2023-03-30
1900. 2023-07-04
1901. 2023-07-19
1902. 2023-02-

3343. 2023-09-29
3344. 2023-06-08
3345. 2023-08-26
3346. 2023-08-14
3347. 2023-08-20
3348. 2023-06-25
3349. 2023-06-18
3350. 2023-08-17
3351. 2023-09-15
3352. 2023-07-06
3353. 2023-07-05
3354. 2023-08-25
3355. 2023-08-07
3356. 2023-06-05
3357. 2023-01-04
3358. 2023-03-16
3359. 2023-01-16
3360. 2023-03-01
3361. 2023-01-18
3362. 2023-09-11
3363. 2023-09-25
3364. 2023-05-27
3365. 2023-07-09
3366. 2023-05-05
3367. 2023-01-17
3368. 2023-09-09
3369. 2023-06-27
3370. 2023-05-09
3371. 2023-01-27
3372. 2023-01-30
3373. 2023-05-19
3374. 2023-08-31
3375. 2023-05-21
3376. 2023-08-20
3377. 2023-08-18
3378. 2023-08-30
3379. 2023-08-15
3380. 2023-04-24
3381. 2023-05-27
3382. 2023-09-02
3383. 2023-03-18
3384. 2023-08-22
3385. 2023-09-15
3386. 2023-09-25
3387. 2023-07-25
3388. 2023-03-28
3389. 2023-07-20
3390. 2023-08-01
3391. 2023-02-14
3392. 2023-06-04
3393. 2023-05-27
3394. 2023-04-08
3395. 2023-01-09
3396. 2023-02-14
3397. 2023-09-10
3398. 2023-03-01
3399. 2023-03-03
3400. 2023-02-05
3401. 2023-07-

4843. 2023-07-21
4844. 2023-07-17
4845. 2023-07-23
4846. 2023-03-10
4847. 2023-07-05
4848. 2023-08-09
4849. 2023-04-06
4850. 2023-06-05
4851. 2023-07-08
4852. 2023-05-20
4853. 2023-05-09
4854. 2023-06-23
4855. 2023-01-07
4856. 2023-01-07
4857. 2023-08-02
4858. 2023-04-10
4859. 2023-05-23
4860. 2023-03-31
4861. 2023-07-31
4862. 2023-01-24
4863. 2023-06-24
4864. 2023-03-22
4865. 2023-07-25
4866. 2023-04-26
4867. 2023-04-03
4868. 2023-07-22
4869. 2023-09-23
4870. 2023-07-29
4871. 2023-09-13
4872. 2023-07-31
4873. 2023-02-08
4874. 2023-05-28
4875. 2023-08-05
4876. 2023-06-02
4877. 2023-08-29
4878. 2023-01-22
4879. 2023-09-28
4880. 2023-09-06
4881. 2023-09-29
4882. 2023-04-02
4883. 2023-06-24
4884. 2023-04-24
4885. 2023-03-28
4886. 2023-02-22
4887. 2023-07-26
4888. 2023-05-27
4889. 2023-04-08
4890. 2023-08-25
4891. 2023-04-21
4892. 2023-08-07
4893. 2023-07-31
4894. 2023-06-07
4895. 2023-07-24
4896. 2023-08-09
4897. 2023-02-17
4898. 2023-10-01
4899. 2023-07-26
4900. 2023-05-16
4901. 2023-08-

In [10]:
dates

['2023-05-04',
 '2023-03-12',
 '2023-08-29',
 '2023-03-09',
 '2023-02-10',
 '2023-02-16',
 '2023-09-08',
 '2023-05-30',
 '2023-07-28',
 '2023-02-09',
 '2023-05-23',
 '2023-04-24',
 '2023-02-23',
 '2023-02-16',
 '2023-06-09',
 '2023-05-17',
 '2023-03-01',
 '2023-09-04',
 '2023-01-03',
 '2023-03-19',
 '2023-08-03',
 '2023-02-17',
 '2023-05-11',
 '2023-03-15',
 '2023-05-29',
 '2023-02-01',
 '2023-08-19',
 '2023-01-30',
 '2023-08-02',
 '2023-06-04',
 '2023-02-16',
 '2023-05-23',
 '2023-05-28',
 '2023-04-20',
 '2023-09-05',
 '2023-03-29',
 '2023-08-15',
 '2023-01-24',
 '2023-05-22',
 '2023-07-09',
 '2023-05-31',
 '2023-09-16',
 '2023-09-09',
 '2023-04-28',
 '2023-02-08',
 '2023-04-23',
 '2023-03-15',
 '2023-07-16',
 '2023-03-25',
 '2023-08-12',
 '2023-09-14',
 '2023-03-14',
 '2023-06-28',
 '2023-06-17',
 '2023-06-14',
 '2023-07-20',
 '2023-07-28',
 '2023-09-14',
 '2023-03-15',
 '2023-05-20',
 '2023-08-23',
 '2023-03-28',
 '2023-05-07',
 '2023-04-24',
 '2023-03-16',
 '2023-01-09',
 '2023-02-

In [28]:
data['Dates'] = dates

In [29]:
data

,target,text,Dates,Email Address
0,0,"Go until jurong point, crazy.. Available only ...",2023-05-04,yNrF0k2r@example.com
1,0,Ok lar... Joking wif u oni...,2023-03-12,yNrF0k2r@example.com
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,2023-08-29,yNrF0k2r@example.com
3,0,U dun say so early hor... U c already then say...,2023-03-09,yNrF0k2r@example.com
4,0,"Nah I don't think he goes to usf, he lives aro...",2023-02-10,yNrF0k2r@example.com
...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,2023-05-02,yNrF0k2r@example.com
5568,0,Will Ì_ b going to esplanade fr home?,2023-05-22,yNrF0k2r@example.com
5569,0,"Pity, * was in mood for that. So...any other s...",2023-08-30,yNrF0k2r@example.com
5570,0,The guy did some bitching but I acted like i'd...,2023-08-29,yNrF0k2r@example.com


In [31]:
import random
import string
emails = []
# Number of email addresses to generate
num_addresses = 5572

# Domain name for the email addresses
domain = "example.com"

# Function to generate a random local part
def generate_local_part():
    local_chars = string.ascii_letters + string.digits
    local_part = ''.join(random.choice(local_chars) for _ in range(8))  # Adjust the length as needed
    return local_part

# Generate random email addresses
email_addresses = [f"{generate_local_part()}@{domain}" for _ in range(num_addresses)]

# Print the result (optional)
for email in email_addresses:
    print(email)
    emails.append(email)

nfEC9L4i@example.com
aFqjKCsB@example.com
V69hAzmH@example.com
MvcKkyFL@example.com
FS0YLjUF@example.com
y0eD3hE4@example.com
MEonYrGE@example.com
MlpKPLBx@example.com
PtLkjyBE@example.com
TUxd80kO@example.com
dah2UfkN@example.com
cveT6Z4R@example.com
aebfqc8e@example.com
dBmlfHhu@example.com
LqCWyEpY@example.com
nfLqeVYR@example.com
OcPT0EmL@example.com
1aQiT3Ir@example.com
NYa7Sw5v@example.com
yr8JvZ82@example.com
prK8ZHNq@example.com
2cKQqaiD@example.com
eue6u3TA@example.com
CmhhsAp9@example.com
9Q6ravlZ@example.com
DcXYYD7K@example.com
yZMdXV3a@example.com
wiAn90fY@example.com
QIg0QpVD@example.com
Wyq5csxW@example.com
7Xg8kJdh@example.com
8NcV0VSP@example.com
yIP5k1MK@example.com
fnd4gQs8@example.com
CC3Vytb3@example.com
YsaVj6ph@example.com
Y2DLW1Cv@example.com
FI1tZhAG@example.com
M7gHIAt1@example.com
WeqGq4yq@example.com
xpQDm5qG@example.com
pew4bSUn@example.com
6tWzQDIx@example.com
Smn41G9b@example.com
3euOFAJo@example.com
CJufl0Xs@example.com
dx2nPTbE@example.com
vxHYgpE4@exam

zAmX4lYX@example.com
Y7MQcy8U@example.com
a3shP65J@example.com
ib2ebZw6@example.com
F1rShkvS@example.com
DKnvpYuu@example.com
CEacq9az@example.com
ctiTGj1n@example.com
S7wovhkT@example.com
g4sglGAB@example.com
wpXzUMQ6@example.com
flFVV4vy@example.com
TvW2eNgT@example.com
OWxgIcqV@example.com
jbDiMDln@example.com
SbSU8dGo@example.com
HgBl567R@example.com
XRCZwvsm@example.com
2io8TUbd@example.com
0qmguUmY@example.com
a7pu5ziP@example.com
mTB6Bozf@example.com
KAUJ4BaO@example.com
UIKME2c3@example.com
Eqswtjd4@example.com
O8eqX8mr@example.com
mGY4sk31@example.com
JUwG0GLB@example.com
4AFvKhre@example.com
cIwd5AvD@example.com
7enOz0rH@example.com
VGktPua0@example.com
slRLIhSQ@example.com
8PsoF0zY@example.com
7TCWdX1w@example.com
JcwnmsCx@example.com
PvfmUSir@example.com
FqLzkDHw@example.com
PgQmbdXU@example.com
vL3dwfVH@example.com
xDA1xHRZ@example.com
Mrp6uvvg@example.com
MF0ufchY@example.com
UgEE20zG@example.com
HUUWWPyu@example.com
b1uqOHLC@example.com
5Pv3iVXs@example.com
y8BbJWvu@exam

8OLvB19V@example.com
fvJIIvda@example.com
xyDZjVdu@example.com
yrZ2B6va@example.com
ZguBy89r@example.com
WEfC3vAm@example.com
IJshFMKZ@example.com
V7H2Bodw@example.com
6cfVnrOP@example.com
5EqfWkZP@example.com
p3lH41tO@example.com
FTOt0KWd@example.com
liRaIW3F@example.com
oZWOdTVe@example.com
cOIphs2v@example.com
VYYehnCg@example.com
WoZf3zaO@example.com
ZIhsHjI4@example.com
wZOaWI0Q@example.com
g64taKYd@example.com
njvwAYww@example.com
sgEPHgrb@example.com
b198F0Lf@example.com
WKMFhzyw@example.com
Y3p2k0jz@example.com
zRqNRxCB@example.com
nTvBDsfO@example.com
XHoEZXvG@example.com
on7h8lYb@example.com
ceSk7tgc@example.com
Zu5YduyS@example.com
3xYh3YfN@example.com
vSi1sxpm@example.com
wl59K1Zk@example.com
viOdOOwX@example.com
rXpUOBuI@example.com
0Qn3hbJD@example.com
HfciEpHN@example.com
QNNy2jaz@example.com
1FzzmGbR@example.com
SPy34jOt@example.com
3nJTnrRV@example.com
EVcBub78@example.com
X6tJNvJU@example.com
QpNCb9qs@example.com
hlwFAjkP@example.com
DNom6Vut@example.com
jyOo7N6f@exam

DSXPVpNj@example.com
PolbHyGx@example.com
wmKRsICb@example.com
CjSGxTWF@example.com
xuQzikTR@example.com
a5jS1s63@example.com
nBFdQQ65@example.com
3Nm4Nz7J@example.com
vtitmKRO@example.com
bQDfGzWy@example.com
J5MX4ufS@example.com
DWHdR78b@example.com
wCqm8Frw@example.com
rfNLyGFB@example.com
4SNFMcp3@example.com
34vaZc2g@example.com
F4uiZNI4@example.com
OuFyETmP@example.com
mW4roKbr@example.com
VvjUiHz2@example.com
vOC5TKIR@example.com
OEZXbcJ9@example.com
OVZkQrQA@example.com
bO63CVHi@example.com
40DuG21h@example.com
V3wWBXM5@example.com
U6IHS9gQ@example.com
94CvZtEv@example.com
VyTDzHyu@example.com
r4YIMcsM@example.com
NVR0ZJAs@example.com
leBsvYK3@example.com
gItqJpAH@example.com
Rn916znD@example.com
KDFDHtmB@example.com
DF5tHbnd@example.com
hlSynCPN@example.com
tA3Z93uV@example.com
N5IGl38K@example.com
hv67fuwI@example.com
Ys1qlU69@example.com
lA7PHj8H@example.com
FhNcxxSX@example.com
ndbyppsI@example.com
C8rnzYHw@example.com
sv9q5TfG@example.com
IXvs4tnM@example.com
xccipzeS@exam

In [32]:
data['Email Address'] = emails

In [33]:
data

,target,text,Dates,Email Address
0,0,"Go until jurong point, crazy.. Available only ...",2023-05-04,nfEC9L4i@example.com
1,0,Ok lar... Joking wif u oni...,2023-03-12,aFqjKCsB@example.com
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,2023-08-29,V69hAzmH@example.com
3,0,U dun say so early hor... U c already then say...,2023-03-09,MvcKkyFL@example.com
4,0,"Nah I don't think he goes to usf, he lives aro...",2023-02-10,FS0YLjUF@example.com
...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,2023-05-02,H1gGKCMP@example.com
5568,0,Will Ì_ b going to esplanade fr home?,2023-05-22,dM6UsFSD@example.com
5569,0,"Pity, * was in mood for that. So...any other s...",2023-08-30,hYBu1cm3@example.com
5570,0,The guy did some bitching but I acted like i'd...,2023-08-29,bhbvtfzu@example.com


In [34]:
data.to_csv("DashboardData.csv")